<a href="https://colab.research.google.com/github/zjsan/selenium-web-scraping/blob/main/Copy_of_selenium_webscraping_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install selenium
!pip install webdriver-manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.4 MB/s eta 0:00:00


In [2]:
!apt-get update
!apt-get install -y chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,113 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,001 kB]
Hit:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:13 http://security.ubuntu.com/ubuntu jammy-security/main

In [58]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import logging
import pandas as pd
import numpy as np

# Set up Chrome WebDriver
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

# Login credentials
username = "jpacang@mmsu.edu.ph"
password = "awanpasswordna"

driver = web_driver()
wait = WebDriverWait(driver, 10)

try:
    driver.get("https://auth.timeshighereducation.com/auth/realms/THE/protocol/openid-connect/auth?client_id=DataPoints&redirect_uri=https%3A%2F%2Fwww.timeshighereducation.com%2Fdatapoints%2F&state=b7eafe6e-0f6f-4b21-8950-dcda25d30a03&response_mode=fragment&response_type=code&scope=openid&nonce=79c86488-b331-4a3a-848b-bceee80cecab")

    el = wait.until(EC.element_to_be_clickable((By.ID, "username")))
    el.send_keys(username)  # Enter username

    el = wait.until(EC.element_to_be_clickable((By.ID, "password")))
    el.send_keys(password)  # Enter password

    el = wait.until(EC.element_to_be_clickable((By.ID, "kc-login")))
    el.click()

    print("Waiting for the profile icon or similar element to confirm login...")
    driver.get("https://www.timeshighereducation.com/datapoints/")
    print(driver.title)#check for successful login
    print("Login successful!")

    element = WebDriverWait(driver, 15).until(
          EC.presence_of_element_located((By.XPATH, "//p[@class='chakra-text css-1qawl6f']"))
      )
    print(element.text)
     #-----Navigating the UI to redirect to the desired page: https://www.timeshighereducation.com/datapoints/sdg/details/1-------

    #click side bar button
    try:
      click_side_bar = WebDriverWait(driver, 10).until(
          EC.element_to_be_clickable((By.XPATH, "//button[@data-testid='menu']"))
      )
      click_side_bar.click()
    except TimeoutException:
      logging.error("Sidebar button not clickable.")

    #clicking sdg impact button
    try:
      click_sdg_impact_btn = driver.find_element(By.XPATH, "//button[@class='chakra-button css-1udhqck']")
      click_sdg_impact_btn.click()
    except TimeoutException:
      logging.error("sdg impact button not clickable")

    #click link to desired web page
    try:
      click_details_link = driver.find_element(By.XPATH, "//ul[@class='css-1ngfogx']/li[2]/a") #finding this elements
      click_details_link.click()#clicking the link to redirect to https://www.timeshighereducation.com/datapoints/sdg/details/1
      print("redirecting to https://www.timeshighereducation.com/datapoints/sdg/details/1")
    except TimeoutException:
      logging.error("link not clickable")

    #if successful redirect
    try:

      # Wait for the iframe to be present
      WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//iframe[@id='ImpactDetails']")))

      # Switch to the iframe
      driver.switch_to.frame(driver.find_element(By.XPATH, "//iframe[@id='ImpactDetails']"))

      element = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.XPATH, "//div[@class='Header__DownloadSectionWrapper-sc-7l4zmc-1 eSouWg']/p"))#find this element in the web page
      )
      print(element.text)
      print("------page redirect success------")
      driver.find_element(By.XPATH, "//div[@class='SubHeader__Wrapper-sc-4m2jl7-0 gvjjWk']/div[2]/div/button[2]").click()#clicking tab-table-button

      element = WebDriverWait(driver, 10).until(
          EC.presence_of_element_located((By.XPATH, "//div[@class='ImpactDetailsCharts__Wrapper-sc-cbsj9y-0 iUEOtz']/div/div/table//td"))#find this element in the web page
      )

      #extracting table headers and data and print them
      headers = driver.find_elements(By.XPATH, "//table[@class='ImpactDetailsTable__Table-sc-y0nk0g-1 yejRQ']//thead")
      table_data = driver.find_elements(By.XPATH, "//table[@class='ImpactDetailsTable__Table-sc-y0nk0g-1 yejRQ']//tbody//tr")

      header1 = []
      header2 = []
      university_name = []
      score = []

      for item in headers: # Only consider the first row for headers
        header1.append(item.find_element(By.XPATH, ".//th[1]").text)
        header2.append(item.find_element(By.XPATH, ".//th[2]").text)
      print(header1)
      print(header2)

      WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//table[@class='ImpactDetailsTable__Table-sc-y0nk0g-1 yejRQ']//td")))

      for item in table_data:
        university_name.append(item.find_element(By.XPATH, ".//td[1]").text)
        score.append(item.find_element(By.XPATH, ".//td[2]").text)
      print(university_name)
      print(score)

      print(len(header1))
      print(len(header2))
      print(len(university_name))
      print(len(score))

      #practicing pandas series
      new_arr = [float(x) for x in score]
      obj = pd.Series(new_arr, index = university_name)
      obj.index.name = "University Name"
      print(obj)

    except Exception as e:
      print(f"Error: {e}")
      print("Element not found")

except Exception as e:
    logging.error(f"Error in login found: {e}")
    print(driver.page_source)
finally:
    driver.quit()

Waiting for the profile icon or similar element to confirm login...
THE DataPoints
Login successful!
Access the powerful data behind THE’s Impact Rankings, based on the UN’s Sustainable Development Goals
redirecting to https://www.timeshighereducation.com/datapoints/sdg/details/1
Data & methodology
------page redirect success------
['University', 'University', 'University', 'University', 'University']
['Score', 'Score', 'Score', 'Score', 'Score']
['Mariano Marcos State University', 'Top - Worldwide', '75% - Worldwide', 'Median - Worldwide', '25% - Worldwide', 'Bottom - Worldwide', 'Mariano Marcos State University', 'Top - Worldwide', '75% - Worldwide', 'Median - Worldwide', '25% - Worldwide', 'Bottom - Worldwide', 'Mariano Marcos State University', 'Top - Worldwide', '75% - Worldwide', 'Median - Worldwide', '25% - Worldwide', 'Bottom - Worldwide', 'Mariano Marcos State University', 'Top - Worldwide', '75% - Worldwide', 'Median - Worldwide', '25% - Worldwide', 'Bottom - Worldwide', 'Mar